In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler
import numpy as np
import subprocess

Alter the file directories appropriately.

In [ ]:
sigdatapath='../input/signal_data.p'
backdatapath='../input/background_data.p'

In [ ]:
sigdata=np.asarray(joblib.load('../input/signal_data.p'))
backdata=np.asarray(joblib.load('../input/background_data.p'))

#Using features 2 and 3

sig_len=len(sigdata)
back_len=len(backdata)

#print len(sigdata),len(backdata)

sigdata=[[e[1],e[2]] for e in sigdata]
backdata=[[e[1],e[2]] for e in backdata]

#print len(sigdata),len(backdata)

train_ratio=0.6
sigtrain=sigdata[:int(train_ratio*sig_len)]
backtrain=backdata[:int(train_ratio*back_len)]
sigtest=sigdata[int(train_ratio*sig_len):]
backtest=backdata[int(train_ratio*back_len):]

#print len(sigtrain),len(backtrain),len(sigtest),len(backtest)

scaler = StandardScaler(copy=True, with_mean=True, with_std=True).fit(sigtrain)
sigtrain = scaler.transform(sigtrain)
sigtest = scaler.transform(sigtest)
backtrain = scaler.transform(backtrain)
backtest = scaler.transform(backtest)

#print len(sigtrain),len(backtrain),len(sigtest),len(backtest)

#print sigtrain

train_data=np.concatenate((sigtrain,backtrain))
test_data=np.concatenate((sigtest,backtest))

#print len(train_data),len(test_data)

train_label=np.zeros(len(train_data))
train_label[:len(sigtrain)]=1
test_label=np.zeros(len(test_data))
test_label[:len(sigtest)]=1

In [ ]:
classifier=MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto',
                           beta_1=0.9, beta_2=0.999, early_stopping=False,
                           epsilon=1e-08, hidden_layer_sizes=(1024,256), learning_rate='constant',
                           learning_rate_init=0.0001, max_iter=100, momentum=0.9,
                           nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
                           solver='adam', tol=0.0001, validation_fraction=0.1, verbose=True,
                           warm_start=False)

In [ ]:
classifier.fit(train_data,train_label)

In [ ]:
train_pred=classifier.predict(train_data)
train_acc=1-abs(sum(train_pred-train_label))/len(train_label)
test_pred=classifier.predict(test_data)
test_acc=1-abs(sum(test_pred-test_label))/len(test_label)
print("Train Accuracy:",train_acc)
print("Test Accuracy:",test_acc)

In [ ]:
#Evaluating figure of merit from training samples.

#Assuming alpha=1
alpha=1

s_tr=len([i for i in range(len(train_pred)) if (train_label[i]==1 and train_pred[i]==1)])
b_tr=len([i for i in range(len(train_pred)) if (train_label[i]==0 and train_pred[i]==0)])
train_fom=alpha*s_tr/(s_tr+b_tr)**0.5

s_te=len([i for i in range(len(test_pred)) if (test_label[i]==1 and test_pred[i]==1)])
b_te=len([i for i in range(len(test_pred)) if (test_label[i]==0 and test_pred[i]==0)])
test_fom=alpha*s_te/(s_te+b_te)**0.5

print("Train FOM:",train_fom)
print("Test FOM:",test_fom)

In [ ]:
_=joblib.dump(classifier,'classifier_Gopalakrishnan')

In [ ]:
print(subprocess.check_output(["ls", "../working"]).decode("utf8"))